In [1]:
import numpy as np
import pandas as pd

### Video Game slaes dataset

In [ ]:
vsg = pd.read_csv("vsales.csv")
vsg

In [ ]:
vsg_cols = vsg[vsg.columns[:6]]
vsg_cols

In [ ]:
vsg_num = vsg[vsg.columns[6:]]
vsg_num

### Example dataset

In [2]:
oneD_arr = np.array([1,2,6,7,8,10,15,17,20]) 

In [3]:
oneD_arr_s = pd.DataFrame({'nums':oneD_arr})

### K-Medoid

In [4]:
dataset = oneD_arr_s.copy()

cols = []

In [5]:
k = 3

centers_indexes = np.random.randint(dataset.index[0],dataset.index[-1] + 1,k)
centers_indexes.sort()
centers_indexes

array([4, 5, 7])

In [6]:
# distance row with df
# ret : np array 
def distances(cset,item):
    dist = np.array([])
    for items in cset.index:
        dist = np.append(dist,np.linalg.norm(cset.loc[items] - item))
    return dist

In [7]:
# distance row with row
# ret : np array
def distance(point_1,point_2):
    return np.linalg.norm(point_1 - point_2)

In [8]:
# check that number in list exist
# ret : index
def in_group(num,lists):
    for i,n in enumerate(lists):
        if num == n:
            return i

In [9]:
def backup(base):
    echo = []
    for df in base:
        echo.append(df.copy())
    return echo

In [10]:
def join_with_cols(csets,cols):
    for df in csets:
        for col in cols:
            temp = []
            for row in df.index:
                temp.append(cols.loc[row][col])
            df[col] = temp

In [11]:
def display(csets):
    for df in csets:
        print(df,"\n")
    print("\nEF : ",stable_ef,"\n")

In [12]:
# calculate distance center item with other item
# ret : list of distance

def distnc_center(dtset,cent):
    distnc = np.ndarray((0,dtset.shape[0]))
    dst = dtset.copy()
    for indx in cent:
        if 'groups' in dtset.columns:
            dst = dtset.drop('groups',axis=1)
        temp = np.array(distances(dst,dst.iloc[indx]),ndmin=2)
        distnc = np.append(distnc,temp,axis=0)
    return distnc

In [ ]:
def distnc_center_2(dtset,cent,dis):
    dst = dtset.drop('groups',axis=1)
    for row in dtset:
        rw = row.drop('groups',axis=1)
        temp = np.array(distances(dst,rw))
        
        for gid in dtset[dtset['groups'] == row['groups']].index:
            

In [13]:
# set group index to items

def grouping(dtset,distnc):
    C_group = np.array([])
    tc = 0
    big_dif = -1
    big_dif_i = -1
    
    for cols_indx in dtset.index:
        min_col = np.min(distnc[:,cols_indx])
        if big_dif == -1 or big_dif < min_col:
            big_dif = min_col
            big_dif_i = cols_indx
        tc += min_col
        C_group = np.append(C_group,in_group(min_col,distnc[:,cols_indx]))
        
    dtset['groups'] = C_group
    return dtset,tc,[big_dif_i,big_dif]

In [ ]:
def move_item_group(dtset,list_d,cent):
    for indx,item in enumerate(cent):
        if distance(dtset.iloc[list_d[0]],item) < list_d[1]:
            dtset['groups'][list_d[0]] = indx
            return indx 

In [ ]:
def update(dtset,cent,which):
    if which == None:
        return cent

    dst = dtset.copy()
    dst = dst[dst['groups'] == which]
    dst = dst.drop('groups',axis=1)
    temp = np.ndarray((0,dst.shape[0]))
    
    for row in dst.index:
        temp = np.append(temp,np.array([distances(dst,dst.loc[row])]),axis=0)
    
    min_cen = -1
    min_cen_i = -1
    for row in dst.index:
        if min_cen == -1 or min_cen > np.sum(temp[row]):
            min_cen = np.sum(temp[row])
            min_cen_i = row
            
    cent[which] = min_cen_i
    return cent

In [ ]:
tc = -1
stable_tc = -1
backup_centers = centers_indexes.copy()
backup_dataset = dataset.copy()

In [ ]:
dataset = backup_dataset.copy()
centers_indexes= backup_centers.copy()

In [ ]:
print(dataset)
print(centers_indexes)

In [ ]:
#print(dataset,"\n\n")

step = 10
while step:
    dis = distnc_center(dataset,centers_indexes)
    
    if 'groups' in dtset.columns:
         distnc_center_2()
    
    dataset,tc,move_item = grouping(dataset,dis)
    
    if stable_tc == -1 or tc < stable_tc:
        stable_tc = tc
        backup_centers = centers_indexes.copy()
        backup_dataset = dataset.copy()
    
    print("distnc :",dis,"\nmove:",move_item)
    
    which = move_item_group(dataset,move_item,centers_indexes)
    print("tc :",tc)
    centers_indexes = update(dataset,centers_indexes,which)
    

    step-=1

In [ ]:
stable_tc

In [ ]:
join_with_cols(backup_dataset,cols)

In [ ]:
display(backup_dataset)

### Method 2

In [ ]:
def manhattan(p1, p2):
    return np.abs((p1[0]-p2[0])) + np.abs((p1[1]-p2[1]))

In [ ]:
def get_costs(data, medoids):
    clusters = {i:[] for i in range(len(medoids))}
    cst = 0
    for d in data:
        dst = np.array([manhattan(d, md) for md in medoids])
        c = dst.argmin()
        clusters[c].append(d)
        cst+=dst.min()

    clusters = {k:np.array(v) for k,v in clusters.items()}
    return clusters, cst

In [ ]:

def KMedoids(data, k, iters = 100):
    medoids = np.array([data[i] for i in range(k)])
    samples,_ = data.shape

    clusters, cost = get_costs(data, medoids)
    count = 0

    colors =  np.array(np.random.randint(0, 255, size =(k, 4)))/255
    colors[:,3]=1

    plt.title(f"Step : 0")
    [plt.scatter(clusters[t][:, 0], clusters[t][:, 1], marker="*", s=100,
                                    color = colors[t]) for t in range(k)]
    plt.scatter(medoids[:, 0], medoids[:, 1], s=200, color=colors)
    plt.show()

    while True:
        swap = False
        for i in range(samples):
            if not i in medoids:
                for j in range(k):
                    tmp_meds = medoids.copy()
                    tmp_meds[j] = i
                    clusters_, cost_ = get_costs(data, tmp_meds)

                    if cost_<cost:
                        medoids = tmp_meds
                        cost = cost_
                        swap = True
                        clusters = clusters_
                        print(f"Medoids Changed to: {medoids}.")
                        plt.title(f"Step : {count+1}")  
                        [plt.scatter(clusters[t][:, 0], clusters[t][:, 1], marker="*", s=100,
                                    color = colors[t]) for t in range(k)]
                        plt.scatter(medoids[:, 0], medoids[:, 1], s=200, color=colors)
                        plt.show()
        count+=1

        if count>=iters:
            print("End of the iterations.")
            break
        if not swap:
            print("No changes.")
            break

In [ ]:
class KMedoidsClass:
    def __init__(self,data,k,iters):
        self.data= data
        self.k = k
        self.iters = iters
        self.medoids = np.array([data[i] for i in range(self.k)])
        self.colors = np.array(np.random.randint(0, 255, size =(self.k, 4)))/255
        self.colors[:,3]=1

In [ ]:
KMedoids(dataset,2)

### K-Medoid On Internet

In [ ]:
#!pip install scikit-learn-extra
#!pip install sklearn_extra
#!python -m pip install sklearn_extra
#!pip install pyclustering
#!pip install kmedoids

In [ ]:
#from sklearn_extra.cluster import KMedoids
#from pyclustering.cluster.kmedoids import kmedoids

# import kkmedoids

In [ ]:
#kmedoids = KMedoids(n_clusters=2, random_state=0).fit(X)